# Import

In [1]:
import pandas as pd
import numpy as np
import pygaze
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy
import glob
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import detectors
import gazeplotter
from collections import defaultdict
# import local lib
import eye_metrics_utils
import data_utils
import gaze_entropy

In [2]:
import warnings
# warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')

In [3]:
csv_files = glob.glob("data/*.csv")

In [4]:
csv_files_one = [v for v in csv_files if "One Gaze-Vergence" in v]
csv_files_two = [v for v in csv_files if "Two Gaze-Vergence" in v]
csv_files_three = [v for v in csv_files if "Three Go-Around Gaze-Vergence" in v]

In [5]:
df_par = pd.read_csv("participant.csv")
group = [df_par[df_par['Group'].str.contains("1")]['ID'].tolist(), df_par[df_par['Group'].str.contains("2")]["ID"].tolist()]
group = [[i[-3:] for i in v] for v in group]
group

[['032', '027', '031', '028', '004', '008', '010', '029', '003', '007', '023'],
 ['021',
  '006',
  '019',
  '022',
  '015',
  '016',
  '014',
  '005',
  '025',
  '002',
  '001',
  '020',
  '011',
  '017']]

In [6]:
def run_all(df_data):
    df_x = df_data.copy()
    if (data_utils.check_percentage_null(df_x) < 0.5): # if missing value > 50%, remove
        return None
    
    time = np.array(df_data['Start Time (secs)'].tolist())

    Efix = eye_metrics_utils.detect_fixations(df_x)
    Eblk = eye_metrics_utils.detect_blinks(df_x)
    Esac = eye_metrics_utils.detect_saccades(df_x)
    Emsac = eye_metrics_utils.detect_microsaccades(df_x)
#     print(Efix)
    X = np.array(Efix).T[3:].T
    Hs, Ht = gaze_entropy.entropy(X)
    total_time = time[-1] - time[0]
    
    return Efix, Hs, Ht, total_time, Eblk, Esac, Emsac
    

In [7]:
feature_groups = []
for g in group:
    trials = []
    for csv_files in [csv_files_one, csv_files_two, csv_files_three]:
        ret = defaultdict(list)
        for csv in csv_files:
            par_id = csv[14:17]
            if par_id not in g:
                continue
                
            print(csv)
            df_data = pd.read_csv(csv)
            print(len(df_data))
            for v in data_utils.data_slicing(df_data, window_length = 2400, stride = 2400):
                r = run_all(v)
                if r != None:
                    Efix, Hs, Ht, total_time, Eblk, Esac, Emsac = r
                    ret["Eblk"].append(Eblk)
                    ret["Efix"].append(Efix)
                    ret["Esac"].append(Esac)
                    ret["Emsac"].append(Emsac)
#                     ret["trans_matrix"].append(trans_matrix)
                    ret["Hs"].append(Hs)
                    ret["Ht"].append(Ht)
                    ret["total_time"].append(total_time)
        trials.append(ret)
    feature_groups.append(trials)

data\PISSS_ID_003_Approach One Gaze-Vergence.csv
9122
data\PISSS_ID_004_Approach One Gaze-Vergence.csv
9307
data\PISSS_ID_007_Approach One Gaze-Vergence.csv
9492
data\PISSS_ID_008_Approach One Gaze-Vergence.csv
9736
data\PISSS_ID_010_Approach One Gaze-Vergence.csv
9554
data\PISSS_ID_023_Approach One Gaze-Vergence.csv
9369
data\PISSS_ID_027_Approach One Gaze-Vergence.csv
9060
data\PISSS_ID_028_Approach One Gaze-Vergence.csv
8999
data\PISSS_ID_029_Approach One Gaze-Vergence.csv
9862
data\PISSS_ID_031_Approach One Gaze-Vergence.csv
9677
data\PISSS_ID_032_Approach One Gaze-Vergence.csv
8629
data\PISSS_ID_003_Approach Two Gaze-Vergence.csv
9368
data\PISSS_ID_004_Approach Two Gaze-Vergence.csv
9862
data\PISSS_ID_007_Approach Two Gaze-Vergence.csv
9677
data\PISSS_ID_008_Approach Two Gaze-Vergence.csv
9923
data\PISSS_ID_010_Approach Two Gaze-Vergence.csv
9923
data\PISSS_ID_023_Approach Two Gaze-Vergence.csv
9677
data\PISSS_ID_027_Approach Two Gaze-Vergence.csv
8568
data\PISSS_ID_028_Approach T

# function

In [8]:
def get_center(clustering, data):
    center = []
    for i in range(len(set(clustering.labels_)) - 1):
        xi = data[np.where(clustering.labels_ == i)]
        cx = sum(xi.T[0])/len(xi)
        cy = sum(xi.T[1])/len(xi)
        center.append((cx,cy))
    return center

# cluster_center = get_center(clustering,X)

In [9]:
def transition_matrix(transitions):
    if len(transitions) == 0:
        return 0
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M


In [10]:
def distance(x,y):
    return ((x[0]-y[0])**2 + (x[1]-y[1])**2)**0.5

def dbscan_predict(cluster_center, X_new, min_dist = 50, metric=distance):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        for i, x_core in enumerate(cluster_center): 
            if metric(x_new, x_core) < min_dist:
                # Assign label of x_core to x_new
                y_new[j] = i
                break

    return y_new


def dbscan_predict2(cluster_center, X_new, min_dist = 50, metric=distance):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        dist = [metric(x_new, v) for v in cluster_center]
#         print("dist", dist)
#         print("argmin", np.argmin(dist))
        if len(dist) == 0:
            continue
        y_new[j] = np.argmin(dist)

    return y_new

In [11]:
def data_slicing(df_data, window_length = 1200, stride = 300):
    L = len(df_data)
    for i in range(L//stride):
        df_slice = df_data.iloc[i*stride:i*stride+window_length].copy()
        if len(df_slice) < 600:
            return
        else:
            yield df_slice

In [12]:
def check_percentage_null(df_data, missing = 0.0):
    return len(df_data[df_data["X Pos"] != 0])/len(df_data)

In [13]:
def run_all(df_data):
    df_data.fillna(0.0, inplace=True)
    if (check_percentage_null(df_data) < 0.5): # if missing value > 50%, remove
        return None

    x = np.array(df_data['X Pos'].tolist())
    y = np.array(df_data['Y Pos'].tolist())
    time = np.array(df_data['Start Time (secs)'].tolist())*1000
    
    # detect blink, fixation and saccade
    Sblk, Eblk = detectors.blink_detection(x,y,time,minlen=6)
    Sfix, Efix = detectors.fixation_detection(x,y,time,maxdist=5,mindur=50)
    Ssac, Esac = detectors.saccade_detection(x,y,time,minlen=5,maxvel=40,maxacc=340)
    
    # clustering
    X = np.array(Efix).T[3:].T
    clustering = DBSCAN(eps=20, min_samples=3, metric = distance).fit(X)
    cluster_center = get_center(clustering,X)
    pred = dbscan_predict2(cluster_center, np.array(Efix).T[3:].T)
    transitions = pred[np.where(pred!=-1)]
    
    # transition matrix and GTE, SGE
    trans_matrix = transition_matrix(transitions)
    Ht = 0
    Hs = 0
    if trans_matrix != 0:
        pA = [len(np.where(np.array(transitions)==i)[0])/len(transitions) for i in range(len(set(transitions)))]
        for i in range(len(pA)):
            Hs += -1 * np.nan_to_num(pA[i]*np.log2(pA[i]))
            t = np.nan_to_num(trans_matrix[i]*np.log2(trans_matrix[i]))
            Ht += -sum(pA[i]*(t))
    
    total_time = time[-1] - time[0]
        
    return Eblk, Efix, Esac, trans_matrix, Hs, Ht, total_time
    

# import data

In [ ]:
csv_files = glob.glob("data/*.csv")

In [ ]:
csv_files_one = [v for v in csv_files if "One Gaze-Left" in v]
csv_files_two = [v for v in csv_files if "Two Gaze-Left" in v]
csv_files_three = [v for v in csv_files if "Three Go-Around Gaze-Left" in v]

In [ ]:
df_par = pd.read_csv("participant.csv")
group = [df_par[df_par['Group'].str.contains("1")]['ID'].tolist(), df_par[df_par['Group'].str.contains("2")]["ID"].tolist()]
group = [[i[-3:] for i in v] for v in group]
group

In [ ]:
feature_groups = []
for g in group:
    trials = []
    for csv_files in [csv_files_one, csv_files_two, csv_files_three]:
        ret = defaultdict(list)
        for csv in csv_files:
            par_id = csv[14:17]
            if par_id not in g:
                continue
                
            print(csv)
            df_data = pd.read_csv(csv)
            print(len(df_data))
            for v in data_slicing(df_data):
                r = run_all(v)
                if r != None:
                    Eblk, Efix, Esac, trans_matrix, Hs, Ht, total_time = r
                    ret["Eblk"].append(Eblk)
                    ret["Efix"].append(Efix)
                    ret["Esac"].append(Esac)
                    ret["trans_matrix"].append(trans_matrix)
                    ret["Hs"].append(Hs)
                    ret["Ht"].append(Ht)
                    ret["total_time"].append(total_time)
        trials.append(ret)
    feature_groups.append(trials)

In [ ]:
df_data = pd.read_csv("data\PISSS_ID_004_Approach Two Gaze-Left.csv")
df_slice = df_data.iloc[0:3000].copy()

df_slice.fillna(0.0, inplace=True)
print(check_percentage_null(df_slice))

x = np.array(df_slice['X Pos'].tolist())
y = np.array(df_slice['Y Pos'].tolist())
time = np.array(df_slice['Start Time (secs)'].tolist())*1000

# detect blink, fixation and saccade
Sblk, Eblk = detectors.blink_detection(x,y,time,minlen=6)
Sfix, Efix = detectors.fixation_detection(x,y,time,maxdist=10,mindur=50)
Ssac, Esac = detectors.saccade_detection(x,y,time,minlen=5,maxvel=40,maxacc=340)


In [67]:
feature_groups[0][0]['Ht']

[1.6398434025011959,
 1.5805701015520255,
 1.3102966864819996,
 1.550397052661859,
 1.7146810082685429,
 1.733488119658769,
 1.703674947551767,
 1.8213769662085535,
 0.9706397086778134,
 1.2883306159845944,
 1.411699874607322,
 0.9054922390470446,
 0.575196474871425,
 1.6126204008974787,
 1.6243385638102594,
 1.3223740167362614,
 1.7243601312041377,
 1.3120864315771419,
 1.5347861101829314,
 0.9712236077151672,
 1.884196795961587,
 1.4196432066351092,
 0.9754979033265575,
 1.6681116287582543,
 2.117453485445392,
 1.6658648437125765,
 1.536587934711609,
 1.275970636253763,
 1.2347547131307393,
 1.394875565232226,
 1.2301361822550603]

# hypothesis test

In [14]:
# standardized effect size - cohen's d 
def effect_size(a, b):
    es = np.abs(np.mean(a) - np.mean(b))
    sd_pooled = np.sqrt((((len(a)-1)*(np.std(a)**2) + (len(b)-1)*(np.std(b)**2)) / (len(a) + len(b) - 2)))
    d = es/sd_pooled
    
    
    return d

In [15]:
def statistic(g):
    print("GROUP 1")
    print("mean trial 1:", np.mean(g[0][0]))
    print("mean trial 2:", np.mean(g[0][1]))
    print("mean trial 3:", np.mean(g[0][2]))

    print("\nstd trial 1:", np.std(g[0][0]))
    print(  "std trial 2:", np.std(g[0][1]))
    print(  "std trial 3:", np.std(g[0][2]))

    print("--------------------------")
    print("GROUP 2")
    print("mean trial 1:", np.mean(g[1][0]))
    print("mean trial 2:", np.mean(g[1][1]))
    print("mean trial 3:", np.mean(g[1][2]))

    print("\nstd trial 1:", np.std(g[1][0]))
    print(  "std trial 2:", np.std(g[1][1]))
    print(  "std trial 3:", np.std(g[1][2]))

In [16]:
def statistic2(g):
    m = np.array([[np.mean(v) for v in u] for u in g]).reshape(-1)
    std = np.array([[np.std(v) for v in u] for u in g]).reshape(-1)
    group = [1,1,1,2,2,2]
    trial = [1,2,3,1,2,3]
    df = pd.DataFrame(zip(group,trial,m,std), columns=['group',"trial","mean","std"])
    return df

In [17]:
def test(g):
    t = []
    p = []
    e = []
    for i in range(3):
        x = scipy.stats.ttest_ind(g[0][i], g[1][i], equal_var = False)
        t.append(x[0])
        p.append(x[1])
        e.append(effect_size(g[0][i],g[1][i]))
        
    return t,p,e

## fixation count

In [18]:
def fixation_rate():
    g = []
    for v in feature_groups:
        x = np.array([len(v) for v in v[0]['Efix']])/np.array(v[0]['total_time'])
        y = np.array([len(v) for v in v[1]['Efix']])/np.array(v[1]['total_time'])
        z = np.array([len(v) for v in v[2]['Efix']])/np.array(v[2]['total_time'])

        g.append([x,y,z])
    return g
g = fixation_rate()

In [19]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.365808,0.319615
1,1,2,1.356372,0.354896
2,1,3,1.541424,0.000000
3,2,1,1.382459,0.279841
4,2,2,1.388468,0.332714
5,2,3,1.464312,0.385446


In [20]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=-0.42033766130011824, pvalue=0.6753869920332292)

In [21]:
effect_size(g[0][0],g[1][0])

0.05644092762465057

## fixation duration

In [22]:
def fixation_duration():

    g = []
    for v in feature_groups:
        x = np.array([])
        for i, p in enumerate(v[0]['Efix']):
            x = np.append(x,np.array(p).T[2])

        y = []
        for i, p in enumerate(v[1]['Efix']):
            y = np.append(y,np.array(p).T[2])

        z = []
        for i, p in enumerate(v[2]['Efix']):
            z = np.append(z,np.array(p).T[2])

        g.append([x,y,z])
        
    return g 
g= fixation_duration()

In [23]:
statistic2(g)

,group,trial,mean,std
0,1,1,0.448595,0.528280
1,1,2,0.463275,0.568367
2,1,3,0.502812,0.448552
3,2,1,0.437502,0.485347
4,2,2,0.490621,0.582826
5,2,3,0.301272,0.233830


In [24]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=-1.5875969972630277, pvalue=0.11244933925890782)

In [25]:
effect_size(g[0][1],g[1][1])

0.0474562683978811

# Blink

In [26]:
def blink_rate():
    g = []
    for v in feature_groups:
        x = np.array([len(v) for v in v[0]['Eblk']])/np.array(v[0]['total_time'])
        y = np.array([len(v) for v in v[1]['Eblk']])/np.array(v[1]['total_time'])
        z = np.array([len(v) for v in v[2]['Eblk']])/np.array(v[2]['total_time'])

        g.append([x,y,z])
    return g
g = blink_rate()

In [27]:
statistic2(g)

,group,trial,mean,std
0,1,1,0.193103,0.174716
1,1,2,0.229910,0.178378
2,1,3,0.000000,0.000000
3,2,1,0.207466,0.146595
4,2,2,0.170704,0.122336
5,2,3,0.154137,0.025701


In [28]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=1.7253979764457643, pvalue=0.08915187096008524)

In [29]:
effect_size(g[0][1],g[1][1])

0.39230827605511503

# Saccade rate

In [30]:
def saccade_rate():
    g = []
    for v in feature_groups:
        x = np.array([len(v) for v in v[0]['Esac']])/np.array(v[0]['total_time'])
        y = np.array([len(v) for v in v[1]['Esac']])/np.array(v[1]['total_time'])
        z = np.array([len(v) for v in v[2]['Esac']])/np.array(v[2]['total_time'])

        g.append([x,y,z])
    return g
g = saccade_rate()

In [31]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.205865,0.550632
1,1,2,1.056649,0.488892
2,1,3,0.924854,0.000000
3,2,1,0.916631,0.242750
4,2,2,0.926029,0.229214
5,2,3,1.130353,0.385422


In [32]:
scipy.stats.levene(g[0][1], g[1][1], center= "mean")

LeveneResult(statistic=8.53178568094892, pvalue=0.004506441779367775)

In [33]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=1.5098854649142264, pvalue=0.13710559652238571)

In [34]:
effect_size(g[0][1],g[1][1])

0.35040668659487595

# Saccade peak

In [35]:
def saccade_peak():

    g = []
    for v in feature_groups:
        x = np.array([])
        for i, p in enumerate(v[0]['Esac']):
            x = np.append(x,np.array(p).T[2])

        y = []
        for i, p in enumerate(v[1]['Esac']):
            y = np.append(y,np.array(p).T[2])

        z = []
        for i, p in enumerate(v[2]['Esac']):
            z = np.append(z,np.array(p).T[2])

        g.append([x,y,z])
        
    return g
g= saccade_peak()

In [36]:
statistic2(g)

,group,trial,mean,std
0,1,1,0.122057,0.042030
1,1,2,0.123503,0.041214
2,1,3,0.120419,0.021798
3,2,1,0.126187,0.036093
4,2,2,0.128550,0.034790
5,2,3,0.111379,0.034709


In [37]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=-3.7552473594484366, pvalue=0.00017632758984188908)

In [38]:
effect_size(g[0][1],g[1][1])

0.1323990198498

# Microsaccades

In [39]:
def msaccade_rate():
    g = []
    for v in feature_groups:
        x = np.array([len(v) for v in v[0]['Emsac']])/np.array(v[0]['total_time'])
        y = np.array([len(v) for v in v[1]['Emsac']])/np.array(v[1]['total_time'])
        z = np.array([len(v) for v in v[2]['Emsac']])/np.array(v[2]['total_time'])

        g.append([x,y,z])
    return g
g = msaccade_rate()

In [40]:
statistic2(g)

,group,trial,mean,std
0,1,1,2.850970,1.610601
1,1,2,2.774702,1.615711
2,1,3,2.466278,0.000000
3,2,1,2.429472,0.870610
4,2,2,2.192312,0.754168
5,2,3,3.018532,0.706677


In [41]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=2.0384687069281777, pvalue=0.046597572974411076)

In [42]:
effect_size(g[0][1],g[1][1])

0.4731682655091028

## SGE

In [43]:
def sge():

    g = []
    for v in feature_groups:
        x = np.array(v[0]['Hs'])
        x=x[x!=0]

        y = np.array(v[1]['Hs'])
        y=y[y!=0]

        z = np.array(v[2]['Hs'])
        z=z[z!=0]

        g.append([x,y,z])
        
    return g

g = sge()

In [44]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.741419,0.480330
1,1,2,1.798354,0.412941
2,1,3,2.736146,0.000000
3,2,1,1.898783,0.597857
4,2,2,1.807851,0.454962
5,2,3,2.180708,0.373570


In [45]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=-0.09905572688970882, pvalue=0.9213363770207129)

In [46]:
effect_size(g[0][1],g[1][1])

0.021782032512491084

## GTE

In [47]:
def gte():
    g = []
    for v in feature_groups:
        x = np.array(v[0]['Ht'])
        x=x[x!=0]

        y = np.array(v[1]['Ht'])
        y=y[y!=0]

        z = np.array(v[2]['Ht'])
        z=z[z!=0]

        g.append([x,y,z])
        
    return g

g= gte()

In [48]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.442276,0.323603
1,1,2,1.446649,0.335394
2,1,3,1.692026,0.000000
3,2,1,1.500653,0.367768
4,2,2,1.464970,0.296407
5,2,3,1.629834,0.125166


In [49]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=-0.26021733047785145, pvalue=0.7953952946016314)

In [50]:
effect_size(g[0][1],g[1][1])

0.058147644954676406

In [51]:
# g = fixation_rate()
# g = fixation_duration()
# g = sge()
# g = gte()
# g = blink_rate()
g = saccade_rate()
# g = msaccade_rate()


In [57]:
m = np.array([[np.mean(v) for v in u] for u in g])
std = np.array([[np.std(v) for v in u] for u in g])
# group = [[1,1,1],[2,2,2]]
trial = [1,2,3]
m0 = [(str(round(v[0], 2)) + "±" + str(round(v[1], 2))) for v in zip(m[0],std[0])]
m1 = [(str(round(v[0], 2)) + "±" + str(round(v[1], 2))) for v in zip(m[1],std[1])]

df = pd.DataFrame(zip(trial,m0,m1,), columns=["trial","group 1 (mean±std)","group 2 (mean±std)"])

t, p, e = test(g)
print(t, p ,e)

df["t"] = t
df['p'] = p
df['effect size'] = e
df

[2.7281668793163063, 1.5098854649142264, nan] [0.009700534184449881, 0.13710559652238571, nan] [0.7510820043929965, 0.35040668659487595, 0.5331785772812805]


,trial,group 1 (mean±std),group 2 (mean±std),t,p,effect size
0,1,1.21±0.55,0.92±0.24,2.728167,0.009701,0.751082
1,2,1.06±0.49,0.93±0.23,1.509885,0.137106,0.350407
2,3,0.92±0.0,1.13±0.39,NaN,NaN,0.533179
